In [1]:
require 'stopwords'
require 'rblearn'
require 'stemmify'
require 'numo/narray'
require 'rblearn'

false

# 70. データの入手・整形

文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [2]:
def knock70
  pos = File.open("../data/rt-polaritydata/rt-polarity.pos").read
  neg = File.open("../data/rt-polaritydata/rt-polarity.pos").read

  pos_tagged = pos.split("\n").map {|line| "+1 #{line}"}
  neg_tagged = neg.split("\n").map {|line| "-1 #{line}"}
  sen = (pos_tagged + neg_tagged).shuffle
  counter = {"+1" => 0, "-1" => 0}

  out = File.open("../data/sentiment.txt", "w")
  sen.each do |line|
    out.puts line
  end
  out.close

  open('../data/sentiment.txt').each_line do |line|
    counter[line[0..1]] += 1
  end

  return counter
end

:knock70

In [3]:
knock70

{"+1"=>5330, "-1"=>5330}

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [4]:
# in order to check if the word is stop word with O(1)
Stop_words = Hash.new {|hash, key| hash[key] = false}
(Stopwords::STOP_WORDS + ["'", ",", '.', ":", ";"]).each {|stop_word| Stop_words[stop_word] = true}

def is_stop?(word)
  Stop_words[word]
end

:is_stop?

# 72. 素性抽出

極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [5]:
def knock72
  labels = []
  features = []

  File.foreach('../data/sentiment.txt') do |line|
    line.chomp!
    label = line[0..1]
    feature = line[3..-1]
    labels << (label == '+1' ? 1 : 0)
    features << feature
  end

  cv = Rblearn::CountVectorizer.new(lambda{|feature| feature.split.map(&:stem).map(&:downcase)}, 1, 0.7)
  cv.fit_transform(features)

  y = Numo::Int8.zeros([labels.size, 1])
  labels.each_with_index do |label, i|
    y[i, 0] = label
  end

  return [cv, y]
end

:knock72

In [6]:
cv, y = knock72

1

1

# 73. 学習

72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [7]:
class LogisticRegressionModel
  attr_accessor :w

  def initialize(eta, threshold)
    @eta = eta
    @threshold = threshold
  end
  
  
  def h(x)
    z = x.dot(@w)
    return 1 / (1 + Numo::NMath.exp(-z))
  end


  def fit(x, y)
    # @w = Numo::Float64.zeros([x.shape[1], 1])
    @w = Numo::Float64.new([x.shape[1], 1]).rand
    60.times do |t|

      # p y-h(x)
      # NOTE: there is something bug
      @w = @w - @eta * x.transpose.dup.dot(h(x) - y) # x.T.dot(h(x) - y): gradient
      puts "#iter: #{t}, accuracy: #{score(x, y)} loss: #{loss(x, y)}" if t % 5 == 0
    end
  end


  def predict(x)
    probabilities = h(x)
    return probabilities > @threshold
  end


  def predict_prob(x)
    return h(x)
  end


  def score(x, y)
    predicted_class = predict(x)
    num_data = predicted_class.shape[0]

    py = predicted_class.to_a
    ry = y.to_a

    num_correct = 0
    num_data.times do |t|
      num_correct += 1 if (py[t] == ry[t])
    end

    return num_correct.to_f / num_data
  end


  def loss(x, y)
    a = - Numo::NMath.log(h(x)).transpose.dot(y)[0, 0]
    b = - Numo::NMath.log(1-h(x)).transpose.dot(1-y)
    (a + b)[0, 0]
  end
end

:loss

In [8]:
x = cv.doc_matrix
x_train, y_train, x_test, y_test = Rblearn::CrossValidation.train_test_split(x, y, 0.3).map(&:dup)
model = LogisticRegressionModel.new(0.01, 0.5)
model.fit(x_train, y_train)

puts "test accuracy: #{model.score(x_test, y_test)}"

#iter: 0, accuracy: 0.5041543822031627 loss: 10730.811830595778
#iter: 5, accuracy: 0.517689627445725 loss: 6306.869565439083
#iter: 10, accuracy: 0.5349772179040472 loss: 5694.257477482135
#iter: 15, accuracy: 0.5483784508174752 loss: 5380.862518014902
#iter: 20, accuracy: 0.5601715357812919 loss: 5188.063445180318
#iter: 25, accuracy: 0.569284374162423 loss: 5055.475991216219
#iter: 30, accuracy: 0.5785312248726883 loss: 4957.42953088436
#iter: 35, accuracy: 0.5876440632538193 loss: 4881.3184616528215
#iter: 40, accuracy: 0.594210667381399 loss: 4820.0381273285475
#iter: 45, accuracy: 0.6006432591798445 loss: 4769.230872131655
#iter: 50, accuracy: 0.6045296167247387 loss: 4726.103535672672
#iter: 55, accuracy: 0.6082819619404986 loss: 4688.795006291941
test accuracy: 0.24202626641651032


# 74. 予測

73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

In [9]:
model.predict(x_test).each_with_index do |arr, index|
  puts arr
  break if index == 30
end

1
0
0
1
0
1
1
0
1
0
0
1
1
0
1
1
0
1
0
0
0
0
1
0
0
1
0
0
1
1
1


# 75. 素性の重み
73で学習したロジスティック回帰モデルの中で，重みの高い素性トップ10と，重みの低い素性トップ10を確認せよ．

In [10]:
feature_names = cv.get_feature_names
weights = model.w.to_a.map{|arr| arr[0]}
name_weights = {}

feature_names.size.times do |t|
  name_weights[feature_names[t]] = weights[t]
end

puts "high"
name_weights.
  sort{|(_, v1), (_, v2)| v2 <=> v1}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

puts "low"
name_weights.
  sort{|(_, v1), (_, v2)| v1 <=> v2}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

1

high
extraordinarili	1.4234735162183512
march	1.3133718251779434
what-if	1.2849130613289004
opera'	1.2735434668402361
abandon	1.2544968153802107
brave	1.2396429960483555
nanook	1.2254701292072554
drug-rel	1.2208203618359303
back-stori	1.2183232544646632
kid-empower	1.1951112456043569
low
world'	-1.164259334017026
common	-1.1508681624202455
group	-1.1011340740216355
difficult	-1.0888586590117568
citi	-0.9901832232428673
invit	-0.979207089251199
pianist	-0.9747608138891163
fantasi	-0.9614735207253302
mention	-0.9348572280644977
sarandon	-0.9201758788422133


1

# 76. ラベル付け

学習データに対してロジスティック回帰モデルを適用し，正解のラベル，予測されたラベル，予測確率をタブ区切り形式で出力せよ．

In [11]:
a = y_test.to_a.map{|arr| arr[0]}
b = model.predict(x_test).to_a.map{|arr| arr[0]}
c = model.predict_prob(x_test).to_a.map{|arr| arr[0]}

output = open('../data/knock76.tsv', 'w')

a.size.times do |t|
  puts "#{a[t]}\t#{b[t]}\t#{c[t]}" if t < 30
  output.puts "#{a[t]}\t#{b[t]}\t#{c[t]}"
end

output.close

1	1	0.6536319565996026
0	0	0.271940265221902
0	0	0.4849206809791339
0	1	0.9024429237125509
1	0	0.23222288503703872
0	1	0.7465775136794623
0	1	0.7099363694623848
1	0	0.45925540414273536
1	1	0.8431491102311728
1	0	0.39818265961313554
0	0	0.18932336431019375
0	1	0.5690796050586355
0	1	0.7329526361808943
1	0	0.3689935710023698
1	1	0.5290729495098113
0	1	0.6558701694514883
1	0	0.31974313798630066
1	1	0.6271283997463941
1	0	0.21539448977047201
1	0	0.4500968705260572
1	0	0.14301422719898657
1	0	0.3807089978675511
0	1	0.6649513989090072
1	0	0.394708319332091
1	0	0.43283838644688366
0	1	0.8734037482858135
1	0	0.433918746839318
1	0	0.40627393525721234
0	1	0.5843112682794569
0	1	0.9238929775830476


# 77. 正解率の計測

76の出力を受け取り，予測の正解率，正例に関する適合率，再現率，F1スコアを求めるプログラムを作成せよ．

In [12]:
num_all = 0
num1 = 0.0
num2 = 0.0
num3 = 0.0

# a is the actual label
# b is the predicted label
File.foreach('../data/knock76.tsv') do |line|
  num_all += 1

  a, b, _ = line.split.map(&:to_f)
  
  if b == 1
    if a == 1
      num1 += 1
    elsif a == 0
      num2 += 1
    end
  elsif b == 0 && a == 1
    num3 += 1
  end
end

precision = num1 / (num1 + num2)
recall = num3 / (num1 + num3)

puts "precision: #{precision}"
puts "recall: #{recall}"
puts "F-1 score: #{2*precision*recall / (precision + recall)}"

precision: 0.2433392539964476
recall: 0.7360308285163777
F-1 score: 0.3657558994859681


# 78. 5分割交差検定

76-77の実験では，学習に用いた事例を評価にも用いたため，正当な評価とは言えない．すなわち，分類器が訓練事例を丸暗記する際の性能を評価しており，モデルの汎化性能を測定していない．そこで，5分割交差検定により，極性分類の正解率，適合率，再現率，F1スコアを求めよ．

In [41]:
kfold = Rblearn::CrossValidation::KFold.new(x.shape[0], 5, true)
model =  LogisticRegressionModel.new(0.01, 0.5)

kfold.create.each do |train_indices, test_indices|
  x_train = x[train_indices, true]
  y_train = y[train_indices, true]

  x_test = x[test_indices, true]
  y_test = y[test_indices, true]
  
  model.fit(x_train, y_train)
  puts "test accuracy: #{model.score(x_test, y_test)}"
end

1

#iter: 0, accuracy: 0.5017589118198874 loss: 12202.52680232824
#iter: 5, accuracy: 0.5082082551594747 loss: 7236.651213714602
#iter: 10, accuracy: 0.5184099437148217 loss: 6591.342166051503
#iter: 15, accuracy: 0.5291979362101313 loss: 6269.9266303445675
#iter: 20, accuracy: 0.5377579737335835 loss: 6077.294943179975
#iter: 25, accuracy: 0.5459662288930581 loss: 5948.93198111173
#iter: 30, accuracy: 0.5513602251407129 loss: 5856.699414554877
#iter: 35, accuracy: 0.5565196998123827 loss: 5786.61470028977
#iter: 40, accuracy: 0.562734521575985 loss: 5731.063864328684
#iter: 45, accuracy: 0.5667213883677298 loss: 5685.590769736164
#iter: 50, accuracy: 0.5689493433395872 loss: 5647.423896535483
#iter: 55, accuracy: 0.5716463414634146 loss: 5614.746518778138
test accuracy: 0.2049718574108818
#iter: 0, accuracy: 0.5003517823639775 loss: 11993.33572467843
#iter: 5, accuracy: 0.5094981238273921 loss: 7216.827084973813
#iter: 10, accuracy: 0.5191135084427767 loss: 6582.678490971446
#iter: 15, a

1